In [1]:
import numpy as np
import pandas as pd

# Similarity Functions

Functions implemented following the paper: "Context-Aware Trace Clustering"
- Substitution Score Calculation
- Insertion Score Calculation
- Distance Calculation

In [2]:
#STEPS for Substitution Score

#STEP 1: Define the symbols in the list of traces
def define_symbols (traces):
    assert type(traces) == list
    symbols = []
    for item in traces:
        symbols.append(set(item))
    x = symbols[0]
    for i in range(len(symbols)):
        x = x.union(symbols[i])
    
    return list(x)


#STEP 2: Define the set of all 3-grams in the logs and their frequencies
def three_grams (traces):
    assert type(traces) == list
    g3 = []
    g3_freq = {}
    for trace in traces:
        for i in range(len(trace)-2):
            g3.append(", ".join(list(trace[i:i+3])))
            try:
                g3_freq[", ".join(list(trace[i:i+3]))] += 1
            except:
                g3_freq[", ".join(list(trace[i:i+3]))] = 1
    return list(set(g3)), g3_freq


#STEP 3: Define the context for symbol a
def define_context(grams):
    
    assert type(grams) == list
    
    context = {}
    for gram in grams:
        x,a,y = gram.split(", ")
        try:
            context[a].append("{0}, {1}".format(x,y))
        except:
            context[a] = []
            context[a].append("{0}, {1}".format(x,y))
            
    #clear dups
    for k in list(context.keys()):
        context[k] = list(set(context[k]))
    
    return context


#STEP 4: define pairs of context
def context_pairs (context):
    
    assert type(context) == dict
    
    context_pairs = {}
    for a in list(context.keys()):
        for b in list(context.keys()):
            if a != b:
                context_pairs["{0}, {1}".format(a, b)] = list(set(context[a]).intersection(set(context[b])))
    
    return context_pairs


#STEP 5: define co-occurrence combinations
def define_cooccurrence(symbols, context_pairs, gram_freq):
    
    assert type(context_pairs) == dict
    assert type(gram_freq) == dict
    assert type(symbols) == list
    
    co_occur = {}
    for k in list(context_pairs.keys()):
        for item in context_pairs[k]:
            for a in symbols:
                for b in symbols:
                    x,y = item.split(", ")[0], item.split(", ")[1]
                    if a == b:
                        try:
                            n = gram_freq["{0}, {1}, {2}".format(x,a,y)]
                            co_occur["{0}, {1}({2}, {3})".format(x,y,a,b)] = (n*(n-1))/2
                        except:
                            co_occur["{0}, {1}({2}, {3})".format(x,y,a,b)] = 0.0
                        
                    elif a != b:
                        try:
                            n_i = gram_freq["{0}, {1}, {2}".format(x,a,y)]
                            n_j = gram_freq["{0}, {1}, {2}".format(x,b,y)]
                            co_occur["{0}, {1}({2}, {3})".format(x,y,a,b)] = n_i*n_j
                        except:
                            co_occur["{0}, {1}({2}, {3})".format(x,y,a,b)] = 0.0
    
    return co_occur


#STEP 6: Define the count of co-occurrences for symbols a,b for all contexts
def co_occur_combos(symbols, con_pairs, co_occurs):
    assert type(symbols) == list
    assert type(con_pairs) == dict
    assert type(co_occurs) == dict
    
    co_occur_combos = {}
    for a in symbols:
        for b in symbols:
            total = 0.0
            for k in list(con_pairs.keys()):
                for item in con_pairs[k]:
                    total += co_occurs["{0}({1}, {2})".format(item,a,b)]
            co_occur_combos["{0}, {1}".format(a,b)] = total
    
    return co_occur_combos


#STEP 7: Define norm on the count of co-occur combos
def define_norm (co_combos):
    assert type(co_combos) == dict
    norm = 0.0
    for k in list(co_combos.keys()):
        norm += co_combos[k]
    
    return norm


#STEP 8: Define matrix M over A x A
def define_matrix (symbols, co_combos, norm):
    assert type(symbols) == list
    assert type(co_combos) == dict
    assert type(norm) == float
    
    mat_M = {}
    for a in symbols:
        for b in symbols:
            mat_M["{0}, {1}".format(a,b)] = co_combos["{0}, {1}".format(a,b)]/norm
    
    return mat_M


#STEP 9: Define the probability of occurrence
def prob_occur (symbols, mat_M):
    assert type(symbols) == list
    assert type(mat_M) == dict
    
    p = {}
    for a in symbols:
        total = 0
        for b in symbols:
            if a != b:
                total += mat_M["{0}, {1}".format(a,b)]
        total += mat_M["{0}, {1}".format(a,a)]
        p["{0}".format(a)] = total
    
    return p


#STEP 10: Define the expected values
def exp_val (symbols, prob):
    assert type(symbols) == list
    assert type(prob) == dict
    
    e_val = {}
    for a in symbols:
        for b in symbols:
            if a == b:
                e_val["{0}, {1}".format(a,b)] = prob["{0}".format(a)]**2
            else:
                e_val["{0}, {1}".format(a,b)] = 2*prob["{0}".format(a)]*prob["{0}".format(b)]
    
    return e_val


#STEP 11: Define the function for substitution scores
def sub_scores (traces):
    assert type(traces) == list
    
    symbols = define_symbols(traces)
    three_gs, three_gs_freq = three_grams(traces)
    cons = define_context(three_gs)
    con_pairs = context_pairs(cons)
    co_occurs = define_cooccurrence(symbols, con_pairs, three_gs_freq)
    co_combos = co_occur_combos(symbols, con_pairs, co_occurs)
    norm = define_norm(co_combos)
    matM = define_matrix(symbols, co_combos, norm)
    probs = prob_occur(symbols, matM)
    e_val = exp_val(symbols, probs)
    
    sub_costs = {}
    for a in symbols:
        for b in symbols:
            if a!=b:
                try:
                    sub_costs["{0}, {1}".format(a,b)] = np.log2(matM["{0}, {1}".format(a,b)]/e_val["{0}, {1}".format(a,b)])
                except:
                    sub_costs["{0}, {1}".format(a,b)] = -1000
    
    return sub_costs

In [3]:
#STEPS 1-3 are the same for Insertion Score

#STEP 4: Define occurence of 3-gram counts
def occ_count (symbols, cons, grams, gfreq):
    assert type(symbols) == list
    assert type(grams) == list
    assert type(cons) == dict
    
    o_counts = {}
    for a in list(cons.keys()):
        for pair in cons[a]:
            x = pair.split(", ")[0]
            y = pair.split(", ")[1]
            o_counts["{0}, {1}({2})".format(x,y,a)] = gfreq["{0}, {1}, {2}".format(x,a,y)]
    
    return o_counts


#STEP 5: define countRgivenL
def countRgL (symbols, ocounts):
    assert type(symbols) == list
    assert type(ocounts) == dict
    
    rgl_counts = {}
    
    for a in symbols:
        for x in symbols:
            #if a !=x:
            total = 0
            for k in list(ocounts.keys()):
                if k.split("(")[0].split(", ")[0] == x and k.split("(")[1] == "{0})".format(a):
                    total += ocounts[k]
            rgl_counts["{0}/{1}".format(a,x)] = total
    
    return rgl_counts


#STEP 6: define norm(a)
def rgl_norm (symbols, rgl_counts):
    assert type(symbols) == list
    assert type(rgl_counts) == dict
    
    rgl_norms = {}
    
    for a in symbols:
        total = 0
        for x in symbols:
            #if a !=x:
            total += rgl_counts["{0}/{1}".format(a,x)]
        rgl_norms["{0}".format(a)] = total
    
    return rgl_norms


#STEP 7: define the probability of all symbols
def rgl_prob (trace):
    assert type(trace) == list
    
    p = {}
    for item in trace:
        for a in item:
            try:
                p["{0}".format(a)] += 1
            except:
                p["{0}".format(a)] = 1
    
    tot_len = 0
    for item in trace:
        tot_len += len(item)
    
    for k in list(p.keys()):
        p[k] = p[k]/tot_len
    
    return p


#STEP 8: define rglNorm
def normed_counts (symbols, rgl, norms):
    assert type(symbols) == list
    assert type(rgl) == dict
    assert type(norms) == dict
    
    normed_rgls = {}
    
    for a in symbols:
        for b in symbols:
            normed_rgls["{0}/{1}".format(a,b)] = rgl["{0}/{1}".format(a,b)]/norms["{0}".format(a)]
    
    return normed_rgls


#STEP 9: Define the function for insertion score
def insert_scores (traces):
    assert type(traces) == list
    
    symbols = define_symbols(traces)
    grams, freq = three_grams(traces)
    cons = define_context(grams)
    oc = occ_count(symbols, cons, grams, freq)
    rgl = countRgL(symbols, oc)
    norms = rgl_norm(symbols, rgl)
    probs = rgl_prob(traces)
    norm_rgls = normed_counts(symbols ,rgl, norms)
    
    scores = {}
    for a in symbols:
        for b in symbols:
            scores["{0}/{1}".format(a,b)] = np.log2(norm_rgls["{0}/{1}".format(a,b)]/probs["{0}".format(a)]*probs["{0}".format(b)])
    
    #replace -inf
    for k in list(scores.keys()):
        if scores[k] == -np.inf:
            scores[k] = -1000
    
    return scores

In [4]:
#function definition for calculating similarity
def calc_similarity(trace1, trace2, sub_cost, ins_cost, probs):
    
    assert type(trace1) == type(trace2) == list
    
    #pad traces
    trace1 = ["_"] + trace1
    trace2 = ["_"] + trace2
    
    #set shorter one as tr1
    if len(trace1) > len(trace2):
        copy = trace1
        trace1 = trace2
        trace2 = copy

    M = len(trace1)
    N = len(trace2)
    sim_table = np.zeros((M,N)) #establish table
    s_score = sub_cost #get substitution score
    ins_score = ins_cost #get insertion score
    p = probs #get probabilities
    
    #fill table, horizontal -> vertical
    for i in range(M):
        for j in range(N):
            
            #original fill horizontal
            if i == 0:
                if j == 0: #first fill
                    sim_table[i][j] = 1000
                elif j == 1: #first insert
                    sim_table[i][j] = p["{0}".format(trace2[j])]
                else: #rest fill, base insert scores
                    sim_table[i][j] = ins_score["{0}/{1}".format(trace2[j], trace2[j-1])] + sim_table[i][j-1]
            
            #original fill vertical
            elif j == 0:
                if i == 0:#first fill
                    sim_table[i][j] = 1000
                elif i == 1:
                    sim_table[i][j] = p["{0}".format(trace1[i])]
                else: #rest fill, base is the opposite of insert scores
                    sim_table[i][j] = -1*ins_score["{0}/{1}".format(trace1[i], trace1[i-1])] + sim_table[i-1][j]
            
            elif trace1[i] == trace2[j]: #no changes
                sim_table[i][j] = sim_table[i-1][j-1]
            
            else: #substitution, insertion or deletion
                
                #determine the min
                op = np.argmax([sim_table[i-1][j], sim_table[i][j-1], sim_table[i-1][j-1]]) #in order, removal, insertion, substitution
                if op == 0:
                    sim_table[i][j] = -1 + sim_table[i-1][j]#-1*ins_score["{0}/{1}".format(trace2[j],trace1[i])] + sim_table[i-1][j] #removal
                elif op == 1:
                    sim_table[i][j] = ins_score["{0}/{1}".format(trace2[j],trace1[i])] + sim_table[i][j-1] #insertion
                elif op == 2:
                    sim_table[i][j] = s_score["{0}, {1}".format(trace1[i],trace2[j])] + sim_table[i-1][j-1] #substitution
                
    return sim_table[i][j] #final score

# LTL-N Sub-conversations

Functions for LTL-N definition, and discovering patterns with long-term dependencies in sequence.

In [5]:
#helper function to count the occurrences of violations to the label
def count_label (sub, labels):
    assert type(sub) == list
    assert type(labels) == list
    
    count = 0
    for item in sub:
        if item not in labels:
            count += 1
    return count

#until-N definition
def until_N (trace, x, y, N):
    
    assert type(trace) == list
    assert type(x) == list
    assert type(y) == list
    
    sol = []
    current = N
    s = -1
    e = -1
    for i in range(len(trace)):
        
        if (trace[i] in x) and e == -1 and s == -1: #finding first instance of x
            s = i
        
        if (s != -1) and (trace[i] not in x) and (trace[i] not in y): #started count and violates Until
            
            if current <= 0: #no more N to give
                s = -1
                e = -1
                current = N
                continue #search for next
                
            else: #more N to give, decrement
                current -= 1
        
        if s != -1 and (trace[i] in y): #found instance of y and x
            e = i
            sol.append((s,e, count_label(trace[s:e],x), e-s)) #append starting and ending index, with number of appearances of x
            s = -1
            e = -1
    
    return sol

#weighting function
def calc_weights(trace, sub_convos):
    
    assert type(trace) == list #takes a list of traces, i.e: event log
    
    tw_id = {}
    center = len(trace)//2
    tf = {}
    idf = {}
    N = 0 #track number of sub_convos

    #instantiate tf and idf values
    for i in range(len(trace)):
        tf[i] = 0
        idf[i] = 0

    #find non-zero from sub_convos table to fill tf and idf table
    for key in list(sub_convos.keys()):
        if sub_convos[key][0] != 0:
            N += 1
            for entry in sub_convos[key][1]: #for each entry
                s = entry[0]
                e = entry[1]
                label1, label2 = key.split(" -> ")

                for i in range(s,e):
                    if trace[i] == label1 or trace[i] == label2:
                        tf[i] += 1
                    else:
                        idf[i] += 1

    #apply log onto idf vals
    for i in range(len(idf)):
        try:
            if idf[i] == 0:
                idf[i] = 0
            else:
                idf[i] = np.log2(N/idf[i])
        except:
            idf[i] = 0

    #calculate weights
    for i in range(len(trace)):
        if center == i:
            tw_id[i] = 1 + (tf[i]*idf[i]) #center
        else:
            tw_id[i] = (1/np.abs(center-i)) + (tf[i]*idf[i]) #w_i + tf-idf(a_i)

    #convert to list
    trace_weights = []
    for k in list(tw_id.keys()):
        trace_weights.append(tw_id[k])
        
    return trace_weights

#weighting function
def calc_weights(trace, sub_convos):
    
    assert type(trace) == list #takes a list of traces, i.e: event log
    
    tw_id = {}
    center = len(trace)//2
    tf = {}
    idf = {}
    N = 0 #track number of sub_convos

    #instantiate tf and idf values
    for i in range(len(trace)):
        tf[i] = 0
        idf[i] = 0

    #find non-zero from sub_convos table to fill tf and idf table
    for key in list(sub_convos.keys()):
        if sub_convos[key][0] != 0:
            N += 1
            for entry in sub_convos[key][1]: #for each entry
                s = entry[0]
                e = entry[1]
                label1, label2 = key.split(" -> ")

                for i in range(s,e):
                    if trace[i] == label1 or trace[i] == label2:
                        tf[i] += 1
                    else:
                        idf[i] += 1

    #apply log onto idf vals
    for i in range(len(idf)):
        try:
            if idf[i] == 0:
                idf[i] = 0
            else:
                idf[i] = np.log2(N/idf[i])
        except:
            idf[i] = 0

    #calculate weights
    for i in range(len(trace)):
        if center == i:
            tw_id[i] = 1 + (tf[i]*idf[i]) #center
        else:
            tw_id[i] = (1/np.abs(center-i)) + (tf[i]*idf[i]) #w_i + tf-idf(a_i)

    #convert to list
    trace_weights = []
    for k in list(tw_id.keys()):
        trace_weights.append(tw_id[k])
        
    return trace_weights

# Pattern-Matching

Functions for specific pattern matching, using the algorithm PrefixSpan

In [6]:
#PrefixSpan variant
def create_init_prefix(trace, min_supp):
    
    assert type(trace) == list
    assert type(min_supp) == int
    
    labels = list(set(trace)) #create the list of symbols
    
    #create initial prefix
    
    #chunk trace for sequences
    sequences = []
    for i in range(0, len(trace), 30):
        sequences.append((i,i+30)) #REFERENCE: sequences[i] = (start, end)
    
    #create bucket for symbols
    bucket = {}
    for label in labels:
        bucket["{0}".format(label)] = 0
    
    #check for support
    for prefix in list(bucket.keys()):
        
        #check all sequences
        for chunk in sequences:
            if prefix in trace[chunk[0]:chunk[1]]: #in the sequence
                bucket["{0}".format(prefix)] += 1
    
    #drop lower than min_supp
    for key in list(bucket.keys()):
        if bucket[key] < min_supp:
            bucket.pop(key)
    
    #create initial projections
    projections = {}
    
    #initialize the list
    for key in list(bucket.keys()):
        projections[key] = []
    
    #populate by searching each sequence
    for item in list(projections.keys()):
        for seq_id in sequences: #should be in the form (start, end)
            #look to replace the start index
            sequence = trace[seq_id[0]:seq_id[1]]
            for i in range(0, len(sequence), len(item)):
                if item == sequence[i]: 
                    projections[item].append((seq_id[0]+i, seq_id[1]))
                    break
    
    #drop entry if projections <= 1
    for key in list(projections.keys()):
        if len(projections[key]) <= 1:
            projections.pop(key)
    
    return bucket, projections

def generate_projections(bucket, trace):
    
    assert type(bucket) == dict #should be in the form {'use.social.convention': 23}
    assert type(trace) == list #should be in the form ['misc', ..., 'use.social.convention']
    
    projections = {}
    sequences = []
    for i in range(0, len(trace), 30):
        sequences.append((i,i+30)) #REFERENCE: sequences[i] = (start, end)
    
    #generate a projection for each bucket item
    
    #initialize the list
    for key in list(bucket.keys()):
        projections[key] = []
    
    #populate by searching each sequence
    for item in list(projections.keys()):
        pref = item.split(", ")
        for seq_id in sequences: #should be in the form (start, end)
            #look to replace the start index
            sequence = trace[seq_id[0]:seq_id[1]]
            for i in range(0, len(sequence), len(pref)):
                if pref == sequence[i:i+len(pref)]: 
                    projections["{0}".format((", ").join(pref))].append((seq_id[0]+i+len(pref)-1, seq_id[1]))
                    break
    
    #drop if not enough
    for k in list(projections.keys()):
        if len(projections[k]) <= 1:
            projections.pop(k)
    
    return projections

def compute_and_drop(projections, trace, min_supp):

    assert type(projections) == dict #should be in the form ("alpha": [(start, end), ... (startN, endN)])
    assert type(trace) == list
    
    labels = list(set(trace))
    proj_list = []
    
    #for every projection, count the i+1 patterns
    for proj in list(projections.keys()):
        #append from the labels to initialize
        iplus1_count = {}
        for L in labels:
            prefix = proj.split(", ") + [L]
            iplus1_count["{0}".format((", ").join(prefix))] = 0
        
            #count the labels
            sequences = projections[proj]
            for seq_id in sequences: #should be in the form (start, end)
                sequence = trace[seq_id[0]:seq_id[1]]
                for i in range(0, len(sequence), len(prefix)):
                    if prefix == sequence[i:i+len(prefix)]:
                        iplus1_count["{0}".format((", ").join(prefix))] += 1
                        break

            #drop
            for k in list(iplus1_count.keys()):
                if iplus1_count[k] < min_supp:
                    iplus1_count.pop(k)
                    
        #output to proj_list
        proj_list.append(iplus1_count)
    
    return proj_list       

#function to loop generate_proj with compute_and_drop
def mine_pattern(trace, min_supp):
    
    assert type(trace) == list
    assert type(min_supp) == int
    
    patts = {}
    docket = []
    doc_ind = 0
    
    #create initial bucket and projections
    init_buq, init_proj = create_init_prefix(trace, min_supp)
    
    docket.append((init_buq, init_proj))
    
    #loop until empty
    while(len(docket) > 0):
        
        curr_buq= docket[0][0] #current bucket
        curr_proj = docket[0][1] #current projections dict for entire bucket

        #compute and drop
        buckets = compute_and_drop(curr_proj, trace, min_supp)

        #for each bucket in buckets
        for bucket in buckets:
            #compute projection for bucket
            next_proj = generate_projections(bucket, trace)
            docket.append((bucket, next_proj))
            
            #add as patt
            for item in bucket:
                try:
                    patts[item] += bucket[item]
                except:
                    patts[item] = bucket[item]
        
        docket.pop(0) #finish item and move on
    
    return patts

def get_ranked_patterns (eventlog, top):
    
    assert type(eventlog) == list
    assert type(top) == int
    
    
    #get ITF counts for all traces
    itf = {}
    for trace in eventlog:
        pattern_count = mine_pattern(trace, 2)
        for k in list(pattern_count.keys()):
            try:
                itf[k] += pattern_count[k]
            except:
                itf[k] = pattern_count[k]

    #apply log(N/itf-count)
    for k in list(itf.keys()):
        itf[k] = np.log(len(itf)/itf[k])

    #calc total PF ITF for each trace patterns
    pf_itf = {}
    for trace in eventlog:
        tf = mine_pattern(trace, 2)
        for k in list(tf.keys()):
            pf_itf[k] = tf[k]*itf[k]

    pf_itf = {k: v for k, v in sorted(pf_itf.items(), key=lambda item: item[1], reverse=True)}

    ranked_patterns = []
    i = 0
    for k in list(pf_itf.keys()):
        ranked_patterns.append((k, pf_itf[k]))
        i += 1

        if i == top:
            break
    
    return ranked_patterns

#modify weights
def augment_weights(trace, trace_weights, rankings):
    
    assert type(trace_weights) == list #should be in the form [weight1, weight2, ..., weightN]
    assert type(rankings) == list #should be in the form [(pattern1, ranking_weight1), ... (patternN, ranking_weightN)]
    
    weights = trace_weights.copy()
    normalize_val = np.sum(weights)
    
    markers = {}
    for item in rankings:
        markers[item[0]] = []
    
    #search in trace for the top 10 rankings
    for item in rankings:
        pattern = item[0].split(", ")
        for i in range(0, len(trace), len(pattern)):
            if pattern == trace[i:i+len(pattern)]:
                markers[item[0]].append((item[1],i, i+len(pattern)-1)) #append (start, end)
    
    #drop empty
    for k in list(markers.keys()):
        if len(markers[k]) < 1:
            markers.pop(k)
    
    #use markers to augment weights
    for k in list(markers.keys()): #search all patterns
        for segment in markers[k]: #for each time pattern occurs
            rank_weight = segment[0]/normalize_val
            start = segment[1]
            end = segment[2]
            #augment
            for q in range(start, end+1):
                weights[q] += rank_weight
    
    return weights

# Load Data

In [ ]:
dat1 = pd.read_csv('./data/graham.norton.s22.e08_data.csv')
dat2 = pd.read_csv('./data/graham.norton.s22.e12_data.csv')
dat3 = pd.read_csv('./data/blackpink_data.csv')
dat4 = pd.read_csv('./data/graham.norton.s22e01.csv')
dat5 = pd.read_csv('./data/graham.norton.s22e02.csv')
dat6 = pd.read_csv('./data/graham.norton.s22e07.csv')
dat7 = pd.read_csv('./data/graham.norton.s22e15.csv')
dat8 = pd.read_csv('./data/graham.norton.s22e19.csv')
dat9 = pd.read_csv('./data/graham.norton.s24e10.csv')
dat10 = pd.read_csv('./data/american_factory.csv')
dat11 = pd.read_csv('./data/taylor_swift_miss_americana.csv')
dat12 = pd.read_csv('./data/spider-man_into_the_spider-verse.csv')

test1 = list(dat1.L) #graham norton
test2 = list(dat2.L)
test3 = list(dat3.L) #blackpink
test4 = list(dat4.L) #graham norton
test5 = list(dat5.L)
test6 = list(dat6.L)
test7 = list(dat7.L)
test8 = list(dat8.L)
test9 = list(dat9.L)
test10 = list(dat10.L) #american factory
test11 = list(dat11.L) #taylor swift
test12 = list(dat12.L) #spider-verse